In [1]:
!pip install spikingjelly
import torch
from torch import nn
import numpy as np
from spikingjelly.activation_based import neuron, functional, surrogate, layer
import torch.nn.functional as F



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
class OverlapPatchMerging(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int, patch_size: int, overlap_size: int
    ):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=patch_size, stride=overlap_size, padding=patch_size // 2, bias=False)
        self.lif = neuron.LIFNode(surrogate_function=surrogate.ATan() ,detach_reset=True, backend='torch', step_mode='m')
        

    def forward(self, x):
        t, b, c, h, w = x.shape
        # print("reached")
        x = self.conv(x.flatten(0,1)) #
        # print(x)
        x = self.lif(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
        # print(x)
        return x

In [3]:
class Mix_FFN(nn.Module):
  '''
  dense layer followed by convolution layer with gelu activation then another
  dense layer
  '''
  def __init__(self, channels, expansion=4):
    super().__init__()
    self.channels = channels
    self.expansion = expansion

    self.dense1 = nn.Conv2d(channels, channels, kernel_size=1)
    self.lif1 = neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')

    self.conv = nn.Conv2d(channels,
                              channels*expansion,
                              kernel_size=3,
                              groups= channels,
                              padding=1)
    self.lif2 = neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')

    self.dense2 = nn.Conv2d(channels*expansion, channels, kernel_size=1)
    self.lif3 = neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')

  def forward(self,x):
    t, b, c, h, w = x.shape
    x=self.dense1(x.flatten(0,1))
    x=self.lif1(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))

    x=self.conv(x.flatten(0,1))
    x=self.lif2(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
    
    x=self.dense2(x.flatten(0,1))
    x=self.lif3(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
    #print("ffn",x.shape)
    return x

In [4]:
import torch
import torch.nn.functional as F
#from config import *

class SSA(nn.Module):
    def __init__(self, d_model, num_heads): #dmodel will be embed dimensions
        super(SSA, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.scale = 0.125

        # Linear transformations for query, key, and value
        self.W_q = nn.Conv1d(d_model, d_model, kernel_size=1, stride=1,bias=False)
        self.q_lif = neuron.LIFNode(tau=2.0, detach_reset=True, step_mode='m', surrogate_function=surrogate.ATan())
        self.W_k = nn.Conv1d(d_model, d_model, kernel_size=1, stride=1,bias=False)
        self.k_lif = neuron.LIFNode(tau=2.0, detach_reset=True, step_mode='m', surrogate_function=surrogate.ATan())
        self.W_v = nn.Conv1d(d_model, d_model, kernel_size=1, stride=1,bias=False)
        self.v_lif = neuron.LIFNode(tau=2.0, detach_reset=True, step_mode='m', surrogate_function=surrogate.ATan())
        # Linear transformation for output
        self.W_o = nn.Conv1d(d_model, d_model, kernel_size=1, stride=1)
        self.o_lif = neuron.LIFNode(tau=2.0, v_threshold=0.5, detach_reset=True, step_mode='m', surrogate_function=surrogate.ATan())
        self.proj= nn.Conv1d(d_model, d_model, stride=1, kernel_size=1)
        self.proj_lif = neuron.LIFNode(tau=2.0, detach_reset=True,step_mode='m', surrogate_function=surrogate.ATan())
       
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate the dot product of query and key matrices
        ktv = torch.matmul(K.transpose(-2, -1),V)
        out= torch.matmul(Q,ktv)*self.scale
        
        # if mask is not None:
        #     scores += mask * -1e9
        
        return out
    
    def forward(self, Q, K, V, t, b):
        # Linear transformations
        #q,k,v are 3d tensors t*b, c, h*w
        Q = self.W_q(Q)
        # print("reached")
        Q = self.q_lif(Q.reshape(t,b,Q.shape[1],Q.shape[2])) 
        # print("reached2")
        K = self.W_k(K)
        K = self.k_lif(K.reshape(t,b,K.shape[1],K.shape[2]))
        V = self.W_v(V)
        V = self.v_lif(V.reshape(t,b,V.shape[1],V.shape[2]))
        
        #4d to 3d
        Q = Q.flatten(0,1)
        K = K.flatten(0,1)
        V = V.flatten(0,1)

        
        # Splitting into multiple heads
        Q = Q.view(Q.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2).contiguous()
        K = K.view(K.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2).contiguous()
        V = V.view(V.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2).contiguous()
        
        # Scaled dot-product attention
        output= self.scaled_dot_product_attention(Q, K, V)
        
        # Concatenate attention outputs of all heads and apply final linear transformation
        output = output.transpose(1, 2).contiguous().view(output.size(0),self.d_model,-1)
        output = self.W_o(output)
        #print("output", output.shape)
        output = self.o_lif(output.reshape(t,b,output.shape[1], output.shape[2]))
        #print("output", output.shape)
        projection = self.proj(output.flatten(0,1))
        #print("proj", projection.shape)
        projection = self.proj_lif(projection.reshape(t,b,projection.shape[1],projection.shape[2]))
        
        # print("testtt", projection.shape)
        
        return projection

In [5]:
# class Rearrange(nn.Module):
#     def forward(self, x):
#         return x.permute(0, 2, 3, 1)

# class RearrangeBack(nn.Module):
#     def forward(self, x):
#         return x.permute(0, 3, 1, 2)

class EfficientMultiHeadedAttention(nn.Module):
  def __init__(self, channels, reduction_ratio, num_heads):
    super().__init__()
    self.conv=nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=reduction_ratio, stride=reduction_ratio)
    self.lif=neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')
    self.att = SSA(d_model = channels, num_heads=num_heads)

  def forward(self, x):
    t, batch_size, chnls, h, w = x.shape
    # print("initial shape",x.shape)
    
    reduced_x=self.conv(x.flatten(0,1))#t*b c h w
    reduced_x = self.lif(reduced_x.reshape(t,batch_size,reduced_x.shape[1],reduced_x.shape[2],reduced_x.shape[3]))#t b c h w
    # print("reduced_x", reduced_x.shape)
    reduced_x = reduced_x.flatten(0,1)#t*b c h w
    reduced_x = reduced_x.reshape(reduced_x.size(0),reduced_x.size(1),-1)#t*b c h*w 
    
    x=x.flatten(0,1)
    x = x.reshape(x.size(0),x.size(1),-1)#t*b c h*w
    #print("final x", x.shape)
    #print("red_x", reduced_x.shape)
    out = self.att(x, reduced_x, reduced_x,t,batch_size)
    # out = out.reshape(out.size(0), out.size(2), h, w)
    # out = out.reshape(out.size(0), out.size(1), h, w)
    out=out.reshape(out.size(0), out.size(1), out.size(2), h, w)
    #print("ema",out.shape)
    
    return out
  

In [6]:
from torchvision.ops import StochasticDepth

class ResidualAdd(nn.Module):
  def __init__(self, fn):
    super().__init__()
    self.fn = fn

  def forward(self, x, **kwargs):
    out = self.fn(x, **kwargs)
    x=x+out
    return x

In [7]:
class SegFormerEncoderBlock(nn.Sequential):
  def __init__(
      self,
      channels: int,
      reduction_ratio: int = 1,
      num_heads: int = 8,
      mlp_expansion: int = 4,
      drop_path_prob: float = 0.0,
  ):

    super().__init__(
        ResidualAdd(
            nn.Sequential(
                EfficientMultiHeadedAttention(channels, reduction_ratio, num_heads),
            )
        ),
        ResidualAdd(
            nn.Sequential(
                Mix_FFN(channels, expansion = mlp_expansion),
                StochasticDepth(p=drop_path_prob, mode="batch")
            )
        ),
    )

In [8]:
from typing import List

class SegFormerEncoderStage(nn.Sequential):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        patch_size: int,
        overlap_size: int,
        drop_probs: List[int],
        depth: int = 2,
        reduction_ratio: int = 1,
        num_heads: int = 8,
        mlp_expansion: int = 4,
    ):
        super().__init__()
        self.overlap_patch_merge = OverlapPatchMerging(
            in_channels, out_channels, patch_size, overlap_size,
        )
        self.blocks = nn.Sequential(
            *[
                SegFormerEncoderBlock(
                    out_channels, reduction_ratio, num_heads, mlp_expansion, drop_probs[i]
                )
                for i in range(depth)
            ]
        )

In [9]:
from typing import Iterable

def chunks(data: Iterable, sizes):
  curr=0
  for size in sizes:
    chunk = data[curr: curr+size]
    curr+=size
    yield chunk


In [10]:
class SegFormerEncoder(nn.Module):
  def __init__(
      self,
      in_channels,
      widths,
      depths,
      all_num_heads,
      patch_sizes,
      overlap_sizes,
      reduction_ratios,
      mlp_expansions,
      drop_prob = 0.0,
  ):
    super().__init__()

    drop_probs = [x.item() for x in torch.linspace(0,drop_prob,sum(depths))]
    self.stages = nn.ModuleList(
        [
            SegFormerEncoderStage(*args)
            for args in zip(
                [in_channels, *widths],
                widths,
                patch_sizes,
                overlap_sizes,
                chunks(drop_probs, sizes=depths),
                depths,
                reduction_ratios,
                all_num_heads,
                mlp_expansions
            )
        ]
    )

  def forward(self,x):
    features = []
    for stage in self.stages:
      x=stage(x)
      features.append(x)
    return features
  


In [11]:
class DecoderBlock(nn.Module):
  def __init__(self, in_channels, out_channels, scale_factor: int =2):
    super().__init__()
    self.upsample=nn.UpsamplingBilinear2d(scale_factor = scale_factor)
    self.conv= nn.Conv2d(in_channels, out_channels, kernel_size = 1)
    self.lif= neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')
  def forward(self,x):
    t, b, c, h, w = x.shape
    x=self.upsample(x.flatten(0,1))
    x=self.conv(x) #b*t *
    x=self.lif(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
    return x

In [12]:
class Decoder(nn.Module):
  def __init__(self, out_channels:int , widths: List[int], scale_factors: List[int]):
    super().__init__()
    self.stages = nn.ModuleList(
        [
            DecoderBlock(in_channels, out_channels, scale_factor)
            for in_channels, scale_factor in zip(widths, scale_factors)
        ]
    )

  def forward(self, features):
    new_features = []
    for feature, stage in zip(features, self.stages):
      x=stage(feature)
      new_features.append(x)

    return new_features


In [13]:
class SegmentationHead(nn.Module):
  def __init__(self, channels, num_classes, num_features = 4):
    super().__init__()
    self.channels = channels
    self.num_classes = num_classes
    self.num_features = num_features
    self.dense1 = nn.Conv2d(channels*num_features, channels, kernel_size=1, bias=False)
    self.lif1= neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')
    self.predict = nn.Conv2d(channels, num_classes, kernel_size=1)
    self.upscale = nn.UpsamplingBilinear2d(scale_factor=4)
    self.lif2 = neuron.LIFNode(surrogate_function=surrogate.ATan(), detach_reset=True, backend='torch', step_mode='m')

  def forward(self,x):
    print(x[0].shape)
    x=torch.cat(x, dim=1)
    t, b, c, h, w = x.shape
    print(x.shape) #3 64 
    x = x.flatten(0,1)
    # x = x.reshape(x.shape[1], x.shape[0], x.shape[2], x.shape[3])
    x=self.dense1(x)
    print('y1')
    x=self.lif1(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
    x=self.predict(x.flatten(0,1))
    x=self.lif2(x.reshape(t, b, x.shape[1], x.shape[2], x.shape[3]))
    x=self.upscale(x)

    return x

In [14]:
class SegFormer(nn.Module):
    def __init__(
        self,
        steps,
        in_channels,
        widths,
        depths,
        all_num_heads,
        patch_sizes,
        overlap_sizes,
        reduction_ratios,
        mlp_expansions,
        decoder_channels,
        scale_factors,
        num_classes,
        drop_prob : float = 0.0,
    ):

        super().__init__()
        self.encoder = SegFormerEncoder(
            in_channels,
            widths,
            depths,
            all_num_heads,
            patch_sizes,
            overlap_sizes,
            reduction_ratios,
            mlp_expansions,
            drop_prob,
        )
        self.decoder = Decoder(decoder_channels, widths[::-1], scale_factors)
        self.seghead = SegmentationHead(
            decoder_channels, num_classes, num_features=len(widths)
        )

        functional.set_step_mode(self,'m')
        #if use_cupy:
        #functional.set_backend(self, backend='cupy')

        self.steps = steps

    def forward(self, x):
        # x = (x.unsqueeze(0)).repeat(self.steps, 1, 1, 1, 1)
        print("img",x.shape)
        features = self.encoder(x)
        print("enc", features[0].shape)
        features = self.decoder(features[::-1])
        print("dec", features[0].shape)
        segmentation = self.seghead(features)
        print("seg", segmentation.shape)
        return segmentation

In [15]:
from torchvision.datasets import Cityscapes
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
# from config import num_steps, batch_size
import torch

from snntorch import spikegen

num_steps = 3
batch_size = 16

#these classes are going to be segmentated total 20 classes
ignore_index = 255
void_classes= [0,1,2,3,4,5,6,9,10,14,15,16,18,29,30,-1]
valid_classes= [ignore_index, 7,8,11,12,13,17,19,20,21,22,23,24,25,26,27,28,31,32,33]
class_names = ['unlabeled',
               'road',
               'sidewalk',
               'building',
               'wall',
               'fence',
               'pole',
               'traffic light',
               'traffic sign',
               'vegetation',
               'terrain',
               'sky',
               'person',
               'rider',
               'car',
               'truck',
               'bus',
               'train',
               'motorcycle',
               'bicycle']
class_map = dict(zip(valid_classes, range(len(valid_classes))))
n_classes = len(valid_classes)

#in later stage we might want to get the segements as different color coded so this is for it
colors =[
    [  0,   0,   0],
    [128,  64, 128],
    [244,  35, 232],
    [220,  20,  60],
    [255,   0,   0],
    [  0,   0, 142],
    [  0,   0,  70],
    [  0,  60, 100],
    [  0,  80, 100],
    [  0,   0, 230],
    [119,  11,  32],
]

label_colors = dict(zip(range(n_classes), colors))

def encode_segmap(mask):
    '''
    online mila tha 
    remove unwanted classes and rectify the labels of wanted classes
    '''
    for void_c in void_classes:
        mask[mask == void_c] = ignore_index
    for valid_c in valid_classes:
        mask[mask == valid_c] = class_map[valid_c]
    
    return mask

def decode_segmap(temp):
    '''
    ye bhi online mila tha
    convert greyscale to color
    ye to use nahi kara hai 
    '''
    temp = temp.numpy()
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0, n_classes):
        r[temp == l] = label_colors[l][0] 
        g[temp == l] = label_colors[l][1] 
        b[temp == l] = label_colors[l][2]
    
    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = r/255.0 
    rgb[:,:,1] = g/255.0 
    rgb[:,:,2] = b/255.0 
    
    return rgb



class AdjustGamma:
    '''
    image bohot dark aa rahi thi to ye laga diya hai
    thoda washed ho gayi hai image par iski wajah se
    '''
    def __init__(self, gamma, gain=1):
        self.gamma = gamma
        self.gain = gain
    
    def __call__(self, image, mask):
        img = np.transpose(image,(2,0,1))
        gamma_tensor = torchvision.transforms.functional.adjust_gamma(torch.from_numpy(img), self.gamma, self.gain)
        img = np.transpose(gamma_tensor.numpy(), (1,2,0))
        return {'image': img, 'mask': mask}

class SpikeEncoding:
    '''
    rate coding ka code
    '''
    def __call__(self, image, mask):
        image = image.float()  # Convert to float
        mask = mask.float()    # Convert to float
        out_img = spikegen.rate(image, num_steps=num_steps)
        out_mask = spikegen.rate(mask, num_steps=num_steps)
        # out_img = out_img.bool()      #IMPORTNAT BUT NOT IMPLEMENTED
        # out_mask = out_mask.bool()
        return {'image': out_img, 'mask': out_mask}
    
class normalizeSeg:
    '''
    segmap me pata nahi normalized values nahi aa rahi thi
    to ye ek alag se bhi bana diya 
    '''
    def __call__(self, image, mask):
        normalized_seg = (mask-torch.min(mask))/(torch.max(mask)-torch.min(mask))
        return {'image': image, 'mask': normalized_seg}

class encode:
    '''
    upar wala encode segmap function use kara hai mask par
    '''
    def __call__(self, image, mask):
        final = encode_segmap(mask)
        return {'image': image, 'mask': final}
'''
yaha par transforms hai 
'''
transform = A.Compose(
    [
        A.Resize(224,224),
        AdjustGamma(gamma=0.63),
        A.Normalize(mean = (0.485, 0.456, 0.406), std= (0.229, 0.224, 0.225), max_pixel_value = float(225)),
        ToTensorV2(),
        encode(),
        normalizeSeg(),
        SpikeEncoding(),
    ]
)


from typing import Any, Callable, Dict, List, Optional, Union, Tuple
from torchvision.datasets import Cityscapes

class data_transform(Cityscapes):
    '''
    isko samajhne ki koi zarurat nahi 
    waise bhi source code uthaya hai bas
    par ye wala dono (image, segmap ) ko return karta hai ek saath
    '''
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        image = Image.open(self.images[index]).convert('RGB')
        
        targets: Any = []
        for i,t in enumerate(self.target_type):
            if t == 'polygon':
                target = self.load_json(self.targets[index][i])
            else:
                target = Image.open(self.targets[index][i])
            targets.append(target)
        target = tuple(targets) if len(targets) > 1 else targets[0]
        

        if self.transforms is not None :
            transformed=transform(image=np.array(image), mask=np.array(target))
            return transformed['image'], transformed['mask']
        return image, target

'''
aise data aa ajyega

dataset = data_transform('/media/iitp/ACER DATA1/cityscapes', split='val', mode='fine', target_type='semantic', transforms=transform)
img, seg = dataset[20]
print(img.shape, seg.shape)

print(img.sum())
print(seg.sum())
print(img.dtype)
print(seg.dtype)

import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML

ch=['r','g','b']
for i in range (0,3):
    img_sample = img[:, i]
    #print(img.size())
    fig, ax = plt.subplots()
    anim = splt.animator(img_sample, fig, ax)
    HTML(anim.to_html5_video())
    anim.save(f"spike_mnist_{ch[i]}.gif")


mask_sample = seg[:]
fig, ax = plt.subplots()
anim = splt.animator(mask_sample, fig, ax)
HTML(anim.to_html5_video())
anim.save(f"mask_sample.gif")
'''

from torch.utils.data import DataLoader
import pytorch_lightning as pl

class CustomCollate:
    def __init__(self, batch_size):
        self.batch_size = batch_size

    def __call__(self, batch):
       # Assuming batch is a list of tensors [image, mask]
       images = [item[0] for item in batch]
       masks = [item[1] for item in batch]
       
       # Stack images and masks along dimension 1
       images_stacked = torch.stack(images, dim=1)
       masks_stacked = torch.stack(masks, dim=1)
       
       return [images_stacked, masks_stacked]


class GetData(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
    
    def setup(self, stage= None):
        self.train_dataset = data_transform(root='../cityscapes', split='train', mode='fine', target_type='semantic', transforms=transform)
        self.val_dataset = data_transform(root='../cityscapes', split='val', mode='fine', target_type='semantic', transforms=transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=CustomCollate(batch_size=batch_size))

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=CustomCollate(batch_size=batch_size))
    



In [16]:
from snntorch import functional as SF
from snntorch import utils
import torch
import torch.utils.tensorboard as tb
import numpy as np
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

GetData_obj = GetData(batch_size=batch_size)
GetData_obj.setup()
train_loader = GetData_obj.train_dataloader()
val_loader = GetData_obj.val_dataloader()


print(f"train_dataloader ready\n number of batches = {len(train_loader)}")
print(f"val_dataloader ready\n number of batches = {len(val_loader)}")

model = SegFormer(
    steps= 4,
    in_channels=3,
    widths=[64, 128, 256, 512],
    depths=[3, 4, 6, 3],
    all_num_heads=[1, 2, 4, 8],
    patch_sizes=[7, 3, 3, 3],
    overlap_sizes=[4, 2, 2, 2],
    reduction_ratios=[8, 4, 2, 1],
    mlp_expansions=[4, 4, 4, 4],
    decoder_channels=256,
    scale_factors=[8, 4, 2, 1],
    num_classes=20,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
num_epochs = 10
loss_fn = SF.ce_rate_loss()

for epoch in range(0, num_epochs):
    start_time = time.time() #i guess it is for showing the final time taken
    model.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0
    for img, label in iter(train_loader):
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)
        spk_rec = model(img)
        print(spk_rec.shape)
        # print(spk_rec)
        print(label.shape)
        

        

        # out_fr = 0
        # for t in range(num_steps): #args.T
        #     out_fr += model(img)

        # out_fr = out_fr / num_steps
        # loss = F.mse_loss(out_fr, label_onehot)
        #loss.backward()
        #optimizer.step()

        #train_samples += label.numel()
        #train_loss += loss.item() * label.numel()
        # The correct rate is calculated as follows. The subscript i of the neuron with the highest firing rate in the output layer is considered as the result of classification.
        #train_acc += (out_fr.argmax(1) == label).float().sum().item()

        # After optimizing the parameters, the state of the network should be reset because the neurons of the SNN have “memory”.
        functional.reset_net(model)

#CHECKPOINTS NOT IMPLEMENTED 

train_dataloader ready
 number of batches = 186
val_dataloader ready
 number of batches = 32
img torch.Size([3, 16, 3, 224, 224])
enc torch.Size([3, 16, 64, 56, 56])
dec torch.Size([3, 16, 256, 56, 56])
torch.Size([3, 16, 256, 56, 56])
torch.Size([3, 64, 256, 56, 56])


RuntimeError: Given groups=1, weight of size [256, 1024, 1, 1], expected input[256, 192, 56, 56] to have 1024 channels, but got 192 channels instead